In [1]:
from rdkit.Chem.rdmolfiles import MolFromPDBFile
from rdkit.Chem.rdchem import Mol
from rdkit.Chem import AllChem

import numpy as np
import rdkit.Chem as Chem
from rdkit.Chem import AddHs, AssignStereochemistry, HybridizationType, ChiralType, BondStereo, MolFromMol2File
from rdkit.Chem.AllChem import ComputeGasteigerCharges
import os
import sys
sys.path.append("../../")
from src.utils.constants import ES_THRESHOLD, METAL_OX_STATES
from tqdm import tqdm 

In [2]:
from src.data.utils import get_ionic_interaction_edge_features

In [3]:
def calculate_gasteiger_charges(mol):
    AllChem.ComputeGasteigerCharges(mol)
    for atom in mol.GetAtoms():
        atom.SetProp('_TriposAtomCharges', str(atom.GetDoubleProp('_GasteigerCharge')))
    return mol


In [4]:
# Define test cases
test_cases = [("CCCCCCCCCC=C", 0, 10), # expected output: None
              ("C=C", 0, 1), # expected output:  (0, 0, 0, 0, 0, 1.3261131312034182)
              ("CC(=O)O", 2, 3) # expected output: (0, 0, 0, 0, 0, 2.278544145756904)
              ]

# Initialize RDKit molecules and conformers
molecules = []
for smiles, a1, a2 in test_cases:
    mol = Chem.MolFromSmiles(smiles)
    AllChem.EmbedMolecule(mol)
    mol = calculate_gasteiger_charges(mol)
    molecules.append((mol, a1, a2))

# Run test cases
for mol, atom_1, atom_2 in molecules:
    # Replace this with the actual function call
    edge_features = get_ionic_interaction_edge_features(mol)
    print(edge_features)

(array([[0.        , 0.        , 0.        , 0.        , 1.48746922],
       [0.        , 0.        , 0.        , 0.        , 1.48746922],
       [0.        , 0.        , 0.        , 0.        , 2.50740432],
       [0.        , 0.        , 0.        , 0.        , 2.50740432],
       [0.        , 0.        , 0.        , 0.        , 3.04999326],
       [0.        , 0.        , 0.        , 0.        , 3.04999326],
       [0.        , 0.        , 0.        , 0.        , 4.42182585],
       [0.        , 0.        , 0.        , 0.        , 4.42182585],
       [0.        , 0.        , 0.        , 0.        , 1.485333  ],
       [0.        , 0.        , 0.        , 0.        , 1.485333  ],
       [0.        , 0.        , 0.        , 0.        , 2.50769486],
       [0.        , 0.        , 0.        , 0.        , 2.50769486],
       [0.        , 0.        , 0.        , 0.        , 3.79374324],
       [0.        , 0.        , 0.        , 0.        , 3.79374324],
       [0.        , 0.        , 0

[17:11:32] Molecule does not have explicit Hs. Consider calling AddHs()
[17:11:32] Molecule does not have explicit Hs. Consider calling AddHs()
[17:11:32] Molecule does not have explicit Hs. Consider calling AddHs()
